In [7]:
#Step 1: import the necessary modules

from __future__ import print_function

from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [8]:
#Step 2, training parameters

batch_size = 64 # batch size (number of training samples/epoch) for training
epochs = 100 # number of epochs to train for
latent_dim = 256 # latent dimensionality of the encoding space
num_samples = 10000 #number of samples (sentences) to train on

In [9]:
#Step 3, File dialog  

#conda install -c anaconda tk 
#https://pythonspot.com/tk-file-dialogs/
#Python3: tkinter, Python2: Tkinter

from tkinter import filedialog
from tkinter import *
 
root = Tk()
root.filename =  filedialog.askopenfilename(initialdir = "/",
                                            title = "Select file",
                                            filetypes = (("Text File","*.txt"),("All Files","*.*")))
print (root.filename)

# path to the data txt file on disk
data_path = root.filename
root.destroy()

C:/Users/3734379/Desktop/fra.txt


In [10]:
#Step 4, vectorize the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

#open the file as read text
with open(data_path, 'r', encoding='utf-8') as f:
    #extract all the lines in a list from the dataset file
    lines = f.read().split('\n')
    
#for each line 
for line in lines[: min(num_samples, len(lines)-1)]: # iterate over first 1000 lines or less
    #from each line extract 2 sentences separated by tab 
    input_text, target_text = line.split('\t')
    
    #tab and new line will be start and end sequences, respectively.
    target_text = '\t' + target_text + '\n'
    
    #append the current input and target sentence to the input and target lists, respectively
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    #construct the character set for input and target
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)

    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

#convert the input character set into a sorted character list, and the same for target set
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

#number of character set elements for input (num enoder tokens), and target (num decoder tokens)
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

#maximum sentence length for input and target
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples (sentences):', len(input_texts))
print('Number of unique input tokens (# unique characters for input):', num_encoder_tokens)
print('Number of unique output tokens (# unique characters for output):', num_decoder_tokens)
print('Max sequence length for inputs (max sentence length for input):', max_encoder_seq_length)
print('Max sequence length for outputs (max sentence length for output):', max_decoder_seq_length)

Number of samples (sentences): 10000
Number of unique input tokens (# unique characters for input): 69
Number of unique output tokens (# unique characters for output): 93
Max sequence length for inputs (max sentence length for input): 16
Max sequence length for outputs (max sentence length for output): 59


In [11]:
#Step 5, inputCharacter-index lookup table
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
print(input_token_index)

#targetCharacters-index lookup table
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
print(target_token_index)

{' ': 0, '!': 1, '$': 2, '%': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '0': 9, '1': 10, '3': 11, '5': 12, '6': 13, '7': 14, '8': 15, '9': 16, ':': 17, '?': 18, 'A': 19, 'B': 20, 'C': 21, 'D': 22, 'E': 23, 'F': 24, 'G': 25, 'H': 26, 'I': 27, 'J': 28, 'K': 29, 'L': 30, 'M': 31, 'N': 32, 'O': 33, 'P': 34, 'Q': 35, 'R': 36, 'S': 37, 'T': 38, 'U': 39, 'V': 40, 'W': 41, 'Y': 42, 'a': 43, 'b': 44, 'c': 45, 'd': 46, 'e': 47, 'f': 48, 'g': 49, 'h': 50, 'i': 51, 'j': 52, 'k': 53, 'l': 54, 'm': 55, 'n': 56, 'o': 57, 'p': 58, 'q': 59, 'r': 60, 's': 61, 't': 62, 'u': 63, 'v': 64, 'w': 65, 'x': 66, 'y': 67, 'z': 68}
{'\t': 0, '\n': 1, ' ': 2, '!': 3, '$': 4, '%': 5, '&': 6, "'": 7, '(': 8, ')': 9, ',': 10, '-': 11, '.': 12, '0': 13, '1': 14, '3': 15, '5': 16, '8': 17, '9': 18, ':': 19, '?': 20, 'A': 21, 'B': 22, 'C': 23, 'D': 24, 'E': 25, 'F': 26, 'G': 27, 'H': 28, 'I': 29, 'J': 30, 'K': 31, 'L': 32, 'M': 33, 'N': 34, 'O': 35, 'P': 36, 'Q': 37, 'R': 38, 'S': 39, 'T': 40, 'U': 41, 'V': 42, 'Y': 43,

In [12]:
#Step 6, turn sentences into 3 numpy arrays
#-----------------------------------

#size(encoder_input_data) = [rows   : max_input_sentence_length,
#                            columns: num_encoder_tokens(characters),
#                            depth  : input_text_length(statements number)]
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')

#size(decoder_input_data) = [rows   : max_target_sentence_length,
#                            columns: num_decoder_tokens(characters),
#                            depth  : input_text_length(statements number)]
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

#size(decoder_target_data) = [rows   : max_target_sentence_length,
#                             columns: num_decoder_tokens(characters),
#                             depth  : input_text_length(statements number)]
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

In [13]:
#for each statement in inputText and targetText
#using the zip() operator
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    
    #for each char in the input statement
    for t, char in enumerate(input_text):
        #[statement number, location in the statement, index of char from dict]
        encoder_input_data[i, t, input_token_index[char]] = 1.
    
    #for each char in the output (target) statement
    for t, char in enumerate(target_text):
        #[statement number, location in the statement, index of char from dict]
        decoder_input_data[i, t, target_token_index[char]] = 1.
        
        #decoder_target_data is ahead of decoder_input_data by one timestep
        if t>0:
            #decoder_target_data will be ahead by one timestep
            #and will not include the start character.
            ##the current character of decoderInputData is the previous character 
            ##of decoderTargetData
            #[statement number, location in the statement - 1, index of char from dict]
            decoder_target_data[i, t-1, target_token_index[char]] = 1.

In [10]:
#Step 7, Construct the LSTM Model
#(define the layer type and shape, then apply data to it)
# Input layer
# LSTM layer
# LSTM(output of Input layer)
# Dense layer
# Dense(output of LSTM layer)

#Encode 
#define an input sequence and process it
encoder_inputs = Input(shape = (None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
#discard the encoderOutputs and kepp the states
encoder_states = [state_h, state_c]

#Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm= LSTM(latent_dim, return_sequences=True, return_state=True)
#set up the decoder, using 'encoder_states' as initial state
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#Model
#define the model that will turn encoderInputData & decoderInputData into decoderTargetData
#the decoder learns to generate targets[t+1 ...] given targets[.. t], conditioned on the input sequence
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data,decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs = epochs,
          validation_split=0.2)
model.save('s2s.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 74s 9ms/step - loss: 0.9188 - val_loss: 0.9504
Epoch 2/100
8000/8000 [==============================] - 77s 10ms/step - loss: 0.7327 - val_loss: 0.7881
Epoch 3/100
8000/8000 [==============================] - 67s 8ms/step - loss: 0.6214 - val_loss: 0.6981
Epoch 4/100
8000/8000 [==============================] - 61s 8ms/step - loss: 0.5639 - val_loss: 0.6436
Epoch 5/100
8000/8000 [==============================] - 65s 8ms/step - loss: 0.5239 - val_loss: 0.6022
Epoch 6/100
8000/8000 [==============================] - 68s 9ms/step - loss: 0.4921 - val_loss: 0.5800
Epoch 7/100
8000/8000 [==============================] - 78s 10ms/step - loss: 0.4661 - val_loss: 0.5537
Epoch 8/100
8000/8000 [==============================] - 60s 7ms/step - loss: 0.4439 - val_loss: 0.5323
Epoch 9/100
8000/8000 [==============================] - 62s 8ms/step - loss: 0.4242 - val_loss: 0.5151
Epoch 10/100
8

C:\Users\3734379\AppData\Local\Continuum\miniconda3\envs\tensorflow\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [14]:
#File dialog to load the trained model  

#conda install -c anaconda tk 
#https://pythonspot.com/tk-file-dialogs/
#Python3: tkinter, Python2: Tkinter

from tkinter import filedialog
from tkinter import *
 
root = Tk()
root.filename =  filedialog.askopenfilename(initialdir = "/",
                                            title = "Select H5 file",
                                            filetypes = (("H5","*.h5"),("All Files","*.*")))
print (root.filename)

# path to the data txt file on disk
data_path = root.filename
root.destroy()    

C:/Users/3734379/Desktop/ABC/Python Codes/s2s.h5


In [15]:
#Inference (testing)
#--------------------
#https://github.com/keras-team/keras/issues/9914
#https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq_restore.py

# Restore the model and construct the encoder and decoder.
model = load_model(data_path)
#----------------------------------------------------------------------------------------
#Encoder
encoder_inputs = model.input[0]   # load input_1 (encoder input) layer
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output # load lstm_1 (encoder_LSTM) layer
encoder_states = [state_h_enc, state_c_enc] #combine the states
# Construct Model using encoderInputLayer and encoderStates (from Encoder LSTM layer)
# These two layers are already trained. 
encoder_model = Model(encoder_inputs, encoder_states)
#---------------------------------------------------------------------------------------
#Decoder
decoder_inputs = model.input[1]   # load input_2 layer

#construct two input layers for the states (acts as a place holder)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))

#combine the outputs of the two Input layers
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm = model.layers[3] #load lstm_2

#use the trained decoderLTSM with (trained decoder Input layer, 2 states Input layers)
#initial_state here is not encoder_states in every time.
#This is because after the first prediction, we fed the states of first prediction
# to the decoderModel
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h_dec, state_c_dec] #combine the states
decoder_dense = model.layers[4] #load the dense layer

decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [16]:
#index-inputCharacter lookup table
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())

#index_targetCharacter llokup table
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [35]:
def decode_sequence(input_seq):
    #Encode the input as state vectors
    #let trained Encoder LSTM generates its states, ignore its o/p  
    states_value = encoder_model.predict(input_seq)
    
    #generate empty target sequence of length 1
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    #populate the first character of target sequence with the start character
    #[1, 0, ..., 0]
    target_seq[0, 0, target_token_index['\t']] = 1.
    
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        #let trained decoder LSTM generates its o/p and states
        output_tokens, h, c = decoder_model.predict([target_seq]+ states_value)
        
        #Sample the token 
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length, or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):

            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]
  
    return decoded_sentence

In [37]:
# Select certain number of statements for testing
n = 100

for seq_index in range(n):
    #take one input sequence (sentence)
    input_seq = encoder_input_data[seq_index:seq_index+1]
    #input_seq = encoder_input_data[seq_index]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input Sentence: ', input_texts[seq_index])
    print('Decoded Sentence: ', decoded_sentence)

-
Input Sentence:  Go.
Decoded Sentence:  Va !

-
Input Sentence:  Hi.
Decoded Sentence:  Salut !

-
Input Sentence:  Run!
Decoded Sentence:  Courez !

-
Input Sentence:  Run!
Decoded Sentence:  Courez !

-
Input Sentence:  Wow!
Decoded Sentence:  Ça alors !

-
Input Sentence:  Fire!
Decoded Sentence:  Au feu !

-
Input Sentence:  Help!
Decoded Sentence:  À l'aide !

-
Input Sentence:  Jump.
Decoded Sentence:  Saute.

-
Input Sentence:  Stop!
Decoded Sentence:  Arrête-toi !

-
Input Sentence:  Stop!
Decoded Sentence:  Arrête-toi !

-
Input Sentence:  Stop!
Decoded Sentence:  Arrête-toi !

-
Input Sentence:  Wait!
Decoded Sentence:  Attendez lus !

-
Input Sentence:  Wait!
Decoded Sentence:  Attendez lus !

-
Input Sentence:  Go on.
Decoded Sentence:  Poursuivez.

-
Input Sentence:  Go on.
Decoded Sentence:  Poursuivez.

-
Input Sentence:  Go on.
Decoded Sentence:  Poursuivez.

-
Input Sentence:  Hello!
Decoded Sentence:  Salut !

-
Input Sentence:  Hello!
Decoded Sentence:  Salut !

-
